In [1]:
import numpy as np

Building treasure map data structure

In [2]:
# build the treasure matrix, the first element is the multiplier, the second is the amount of hunters

row0 = [
    [24, 2],
    [70, 4],
    [41, 3],
    [21, 2],
    [60, 4]
]

row1 = [
    [47, 3],
    [82, 5],
    [87, 5],
    [80, 5],
    [35, 4]
]

row2 = [
    [73, 4],
    [89, 5],
    [100, 8],
    [90, 7],
    [17, 2]
]

row3 = [
    [77, 5],
    [83, 5],
    [85, 5],
    [79, 5],
    [55, 4]
]

row4 = [
    [12, 2],
    [27, 3],
    [52, 4],
    [15, 2],
    [30, 3]
]

In [3]:
treasures = np.array([row0, row1, row2, row3, row4])

In [4]:
treasures.shape

(5, 5, 2)

In [5]:
# the value of one treasure is 7500
multiplier_value = 7500

# cost of choosing to get the jth treasure, with a max of 3 treasures per player
costs = np.array([0, 25_000, 75_000])

In [6]:
# just checking how many treasures can possibly be the third treasure
treasures[:,:,0] * multiplier_value / (treasures[:,:,1]) > 75_000

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True, False],
       [ True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [False, False,  True, False, False]])

creating players

In [7]:
# # pd = player distribution

# # adds the final step at the end to make it easy to sum to 100, therefore will always have 1 more
# # element than the number of steps input

# # the ith element of the list is the percentage of people in the ith step of logic
# # example: pd = [25, 50, 25] means 25% of people are in step 0, 50% in step 1, and 25% in step 2
# pd = [30, 30, 20, 10, 15]

# pd.append(100 - sum(pd))
# if pd[-1] < 0:
#     raise ValueError("Sum of pd is greater than 100")

In [8]:
np.set_printoptions(precision=2)

# define util function to nicely print the array
def print_array(array):
    for row in array:
        print(row)

In [9]:
def choose_treasure(expected_values):
    # find the best 3 treasures to choose
    evs = expected_values.reshape(-1)
    top3 = np.argsort(evs)[-3:][::-1] # reverse order of the top 3 values
    top3_values = evs[top3]
    choices = []
    for i in range(3):
        if top3_values[i] - costs[i] > 0:
            choices.append(top3[i])
    return choices

In [10]:
# get the expected value of all the treasures in the map given the player's assumption of everyone's choices
def get_evs(treasures, assumed_choices):
    # treasures and choices are 5x5 matrices
    island_values =  treasures[:,:,0] * multiplier_value
    number_of_hunters = treasures[:,:,1] + assumed_choices
    evs = island_values / number_of_hunters
    return evs

random_choices = np.ones((5,5)) * 4
evs = get_evs(treasures, random_choices)
print_array(evs)

[30000.   65625.   43928.57 26250.   56250.  ]
[50357.14 68333.33 72500.   66666.67 32812.5 ]
[68437.5  74166.67 62500.   61363.64 21250.  ]
[64166.67 69166.67 70833.33 65833.33 51562.5 ]
[15000.   28928.57 48750.   18750.   32142.86]


In [11]:
print_array(choose_treasure(evs))

11
7


In [12]:
# not right maybe idk
def get_next_choices(treasures, assumed_choices):
    evs = get_evs(treasures, assumed_choices)
    choices = choose_treasure(evs) # ?
    new_choices = 100 * evs / evs.sum()
    return new_choices
    

In [13]:
choices = np.ones((5,5)) * 4
print_array(treasures[:,:,0] * multiplier_value / (treasures[:,:,1] + choices))

[30000.   65625.   43928.57 26250.   56250.  ]
[50357.14 68333.33 72500.   66666.67 32812.5 ]
[68437.5  74166.67 62500.   61363.64 21250.  ]
[64166.67 69166.67 70833.33 65833.33 51562.5 ]
[15000.   28928.57 48750.   18750.   32142.86]


In [57]:
choices = np.ones((5,5)) * 4

for i in range(100):
    choices = get_next_choices(treasures, choices)

In [58]:
print_array(choices)

[2.93 4.8  3.69 2.7  4.34]
[4.03 4.96 5.16 4.88 3.01]
[4.93 5.24 4.73 4.66 2.35]
[4.76 5.   5.08 4.84 4.1 ]
[1.87 2.8  3.95 2.17 3.01]


In [59]:
final_evs = get_evs(treasures, choices)
final_choices = choose_treasure(final_evs)
print(final_evs)
print(final_choices)


[[36486.78 59675.8  45941.08 33537.7  53966.33]
 [50137.25 61730.25 64209.64 60719.75 37444.83]
 [61314.92 65183.51 58894.2  57911.56 29282.58]
 [59182.8  62231.4  63225.7  60210.31 50950.56]
 [23257.39 34886.08 49082.09 26983.17 37438.71]]
[11, 7]


# playing around with assumed distributions

In [88]:
flat_scores = (treasures[:,:,0] * multiplier_value / treasures[:,:,1]).flatten()
ids = np.argsort(flat_scores)
print(flat_scores)

middle = flat_scores[ids[3:-3]] # middle 19 values
print(middle.shape)
print(middle)

[ 90000.   131250.   102500.    78750.   112500.   117500.   123000.
 130500.   120000.    65625.   136875.   133500.    93750.    96428.57
  63750.   115500.   124500.   127500.   118500.   103125.    45000.
  67500.    97500.    56250.    75000.  ]
(19,)
[ 65625.    67500.    75000.    78750.    90000.    93750.    96428.57
  97500.   102500.   103125.   112500.   115500.   117500.   118500.
 120000.   123000.   124500.   127500.   130500.  ]


In [79]:
greedy3 = 10 * np.ones((3,))

agreedy3 = (20/3) * np.ones((3,)) # anti greedy 3

In [85]:
greedy3 = np.array([10, 10, 10])

agreedy3 = np.array([10/3, 10/3, 10/3]) # anti greedy 3

In [86]:
scores = 100 * middle / middle.sum() # get percentage value of each score
scores = scores * ((100 - greedy3.sum() - agreedy3.sum()) / 100)
print(scores)


[2.01 2.07 2.3  2.41 2.76 2.87 2.95 2.99 3.14 3.16 3.44 3.54 3.6  3.63
 3.67 3.77 3.81 3.9  4.  ]


In [89]:
my_assumption = np.ones((25))
my_assumption[ids[:3]] = agreedy3
my_assumption[ids[-3:]] = greedy3
my_assumption[ids[3:-3]] = scores
my_assumption = my_assumption.reshape((5,5))
print(my_assumption)

[[ 2.76 10.    3.14  2.41  3.44]
 [ 3.6   3.77  4.    3.67  2.01]
 [10.   10.    2.87  2.95  3.33]
 [ 3.54  3.81  3.9   3.63  3.16]
 [ 3.33  2.07  2.99  3.33  2.3 ]]


In [90]:
print(my_assumption.sum())

100.0


In [91]:
evs = get_evs(treasures, my_assumption)
print_array(evs)
print_array(choose_treasure(evs))

[37850.48 37500.   50095.55 35705.3  60447.78]
[53429.1  70158.04 72535.84 69171.67 43682.6 ]
[39107.14 44500.   68994.9  67822.98 23906.25]
[67652.31 70643.51 71599.44 68670.6  57632.62]
[16875.   39967.11 55832.46 21093.75 42482.53]
7
17
